In [67]:
# TensorFlow and TF-Hub modules.
from absl import logging

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

from collections import defaultdict


logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the dataset.
import pandas as pd
import numpy as np

# Some modules to display an animation using imageio.
import imageio
from IPython import display


In [72]:
class DeepFakeDetectionNeuralNetwork():
    def __init__(self, input_shape, batch_size=32):
        self.input_shape = input_shape
        self.model = self.create_model()
        self.batch_size = batch_size
        self.model.compile(loss='binary_crossentropy',
                    optimizer=Adam(learning_rate=0.001),
                    metrics=['accuracy'])
        self.predictions = None
        
        # Create the image generators
        self.train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')
        
        self.val_datagen = ImageDataGenerator(rescale=1./255)
        
        # Create the image generators
        self.train_generator = self.train_datagen.flow_from_directory(
            'train_frames/',  
            target_size=(150, 150),
            batch_size=self.batch_size,
            class_mode='binary')
        
        self.val_generator = self.val_datagen.flow_from_directory(
            'train_frames/',  
            target_size=(150, 150), 
            batch_size=self.batch_size,
            class_mode='binary')
     
    def create_model(self):
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu',
                            input_shape=self.input_shape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))    
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        return model
    
    def train_model(self, epochs=10):
        self.model.fit(
            self.train_generator,
            epochs=epochs,
            validation_data=self.val_generator,
            )
        self.model.save(f"{self.__class__.__name__}{self.batch_size}.h5")
        
    def predict(self):
        test_datagen = ImageDataGenerator()

        test_generator = test_datagen.flow_from_directory(
            'test_frames/',
            target_size=(150, 150),
            shuffle=False,
            batch_size=1,
            class_mode='binary')

        filenames = test_generator.filenames
        n = len(filenames)

        pred = deepfakedetection.model.predict_generator(test_generator, steps=n)
        
        vidoes = defaultdict(list)
        for i, path in enumerate(filenames):
            vid = path.split("\\")[1]
            vidoes[vid].append(i)

        predictions = {}
        for vid in vidoes:
            predictions[vid] = np.mean(pred[vidoes[vid]])
        
        return predictions
    
    def get_accuracy(self, predictions):
        
        # round predictions dict to 0 or 1
        predictions = {k: int(round(v)) for k, v in predictions}
        predictions
        # Check the accuracy of the model by comparing the predictions to the ground truth. 0 is fake, 1 is real.
        labels = pd.read_json('test_frames/metadata.json')

        correct = 0
        for k in predictions.keys():
            if labels[k]['label'] == "FAKE":
                if predictions[k] == 1:
                    correct += 1
            else:
                if predictions[k] == 0:
                    correct += 1

        accuracy = correct / len(predictions)
        return accuracy

In [69]:
deepfakedetection = DeepFakeDetectionNeuralNetwork((150, 150, 3), 64)
deepfakedetection.train_model()

Found 119974 images belonging to 2 classes.
Found 119974 images belonging to 2 classes.
Epoch 1/10
1875/1875 [==============================] - 2862s 2s/step - loss: 0.4430 - accuracy: 0.8102 - val_loss: 0.4101 - val_accuracy: 0.8174
Epoch 2/10
1875/1875 [==============================] - 2709s 1s/step - loss: 0.4107 - accuracy: 0.8179 - val_loss: 0.3875 - val_accuracy: 0.8206
Epoch 3/10
1875/1875 [==============================] - 2657s 1s/step - loss: 0.3951 - accuracy: 0.8213 - val_loss: 0.4018 - val_accuracy: 0.8221
Epoch 4/10
1875/1875 [==============================] - 2651s 1s/step - loss: 0.3850 - accuracy: 0.8253 - val_loss: 0.3646 - val_accuracy: 0.8308
Epoch 5/10
1875/1875 [==============================] - 2675s 1s/step - loss: 0.3780 - accuracy: 0.8266 - val_loss: 0.3725 - val_accuracy: 0.8264
Epoch 6/10
1875/1875 [==============================] - 2692s 1s/step - loss: 0.3678 - accuracy: 0.8287 - val_loss: 0.3540 - val_accuracy: 0.8307
Epoch 7/10
1875/1875 [==============

In [71]:
predictions = deepfakedetection.predict()

Found 24268 images belonging to 2 classes.


C:\Users\Dvalv\AppData\Local\Temp/ipykernel_10784/3771048304.py:73: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = deepfakedetection.model.predict_generator(test_generator, steps=n)


In [78]:
# round predictions dict to 0 or 1
predictions = {k: int(round(v)) for k, v in predictions}
predictions


ValueError: too many values to unpack (expected 2)

0.74

In [80]:
deepfakedetection_128 = DeepFakeDetectionNeuralNetwork((150, 150, 3), 32)
deepfakedetection_128.train_model(10)

Found 119974 images belonging to 2 classes.
Found 119974 images belonging to 2 classes.
Epoch 1/10
3750/3750 [==============================] - 2942s 784ms/step - loss: 0.4463 - accuracy: 0.8100 - val_loss: 0.4052 - val_accuracy: 0.8183
Epoch 2/10
3750/3750 [==============================] - 2953s 787ms/step - loss: 0.4117 - accuracy: 0.8169 - val_loss: 0.3904 - val_accuracy: 0.8230
Epoch 3/10
3750/3750 [==============================] - 2953s 787ms/step - loss: 0.4010 - accuracy: 0.8198 - val_loss: 0.3888 - val_accuracy: 0.8233
Epoch 4/10
3750/3750 [==============================] - 2983s 795ms/step - loss: 0.3892 - accuracy: 0.8221 - val_loss: 0.3782 - val_accuracy: 0.8276
Epoch 5/10
3750/3750 [==============================] - 2971s 792ms/step - loss: 0.3800 - accuracy: 0.8245 - val_loss: 0.3732 - val_accuracy: 0.8189
Epoch 6/10
2448/3750 [==================>...........] - ETA: 12:29 - loss: 0.3763 - accuracy: 0.8259